In [109]:
# load up the test file
data = open("Test.neu").read()

# deal with newlines
data = data.split("\n")
for i, line in enumerate(data):
    data[i] = line.strip()
data = "".join(data)

data

'@{string,float,int}#{2:"100"}*{&0,0:"Hello, world! :@#*",1:"512.0",&0}'

In [110]:
cache = {
    "names": [],
    "cache": [],
    "objects": []
}

# parse a string entry according to its type and return a dict
def parse_entry(entry_type, entry_value):
    # convert values to proper types as needed
    if entry_type == "float":
        converted_value = float(entry_value)
    elif entry_type == "int":
        converted_value = int(entry_value)
    else:
        converted_value = entry_value

    # return a dictionary of the parsed entry
    return {"type": entry_type, "value": converted_value}

# iterate through each character in the raw data and create a cache of parsed strings
stage = ""
entry_type = ""
entry_value = ""
consume = False
is_literal = False
is_reference = False
for char in data:
    if char == '"':
        # enter/exit literal consumption (handle special characters in values)
        is_literal = not is_literal
        continue
    # ignore special characters if they're within a literal value
    if not is_literal:
        if char == "@":
            stage = "names"
            continue
        elif char == "*":
            stage = "objects"
            continue
        elif char == "#":
            stage = "cache"
            continue
        elif char == "{":
            # start consuming at the start of each stage
            consume = True
            continue
        elif char == "}":
            # commit entry
            if stage != "names":
                cache[stage].append(parse_entry(entry_type, entry_value))
            else:
                cache[stage].append(entry_value)
            entry_value = ""

            is_reference = False

            # stop consuming at the end of each stage
            consume = False
        elif char == ",":
            # commit entry
            if stage != "names":
                cache[stage].append(parse_entry(entry_type, entry_value))
            else:
                cache[stage].append(entry_value)
            entry_value = ""
            
            is_reference = False

            continue
        elif char == ":":
            # handle names/types
            entry_type = cache["names"][int(entry_value, base = 16)]
            entry_value = ""
            continue

    if consume:
        # handle cache references
        if entry_value == "&":
            cached_object = cache["cache"][int(char, base = 16)]
            entry_type = cached_object["type"]
            entry_value = cached_object["value"]
            continue
        else:
            entry_value += char

In [111]:
graph = cache["objects"]

In [112]:
# DEBUG
import json
with open("NeutrinoTest_Cache.json", "w") as outfile:
    json.dump(graph, outfile, indent = 4)